# ReadExtractIMDBTitleInfo

In [23]:
import os
import re
import urllib
import tarfile
import sqlite3
import gzip
import csv

import pandas as pd

## Download Data

In [2]:
os.makedirs("var", exist_ok=True)

In [3]:
data_url = "https://datasets.imdbws.com/title.basics.tsv.gz"
local_file = "var/title.basics.tsv.gz"

In [4]:
if not(os.path.isfile(local_file)):
    urllib.request.urlretrieve(data_url, local_file)

## Load Data

In [35]:
title_info = pd.read_csv(local_file, sep='\t', encoding='utf-8', na_values=['\\N'], quoting=csv.QUOTE_NONE)

In [36]:
title_info.sample(5)

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres
7204868,tt6437316,short,The Moment,The Moment,0,2016.0,NaN,2.0,"Animation,Drama,Short"
1634985,tt11193208,tvEpisode,PIZZAMAS IS HERE!!!!,PIZZAMAS IS HERE!!!!,0,2019.0,NaN,NaN,"Comedy,Documentary,Music"
6984716,tt5943638,tvEpisode,Your Vibe Attracts Your Tribe,Your Vibe Attracts Your Tribe,0,2016.0,NaN,11.0,Comedy
5736532,tt3110852,tvEpisode,The Evolution of Funny: TV Comedy with Larry W...,The Evolution of Funny: TV Comedy with Larry W...,0,2013.0,NaN,27.0,Talk-Show
2276283,tt12349736,short,Licença Poética,Licença Poética,0,2019.0,NaN,13.0,Short


In [37]:
title_info.isAdult.value_counts().to_frame()

,isAdult
0,8527053
1,272365


In [38]:
title_info.titleType.value_counts().to_frame()

,titleType
tvEpisode,6594590
short,862074
movie,605618
video,258503
tvSeries,222707
tvMovie,135520
tvMiniSeries,42742
tvSpecial,36408
videoGame,30742
tvShort,10512


In [39]:
title_info.startYear.value_counts().to_frame()

,startYear
2018.0,411707
2017.0,409565
2019.0,396631
2016.0,389624
2021.0,384671
...,...
1882.0,2
1881.0,2
1874.0,1
1883.0,1


In [40]:
title_info = title_info.rename(columns={'tconst': 'titleId'})

In [41]:
title_info

,titleId,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres
0,tt0000001,short,Carmencita,Carmencita,0,1894.0,NaN,1.0,"Documentary,Short"
1,tt0000002,short,Le clown et ses chiens,Le clown et ses chiens,0,1892.0,NaN,5.0,"Animation,Short"
2,tt0000003,short,Pauvre Pierrot,Pauvre Pierrot,0,1892.0,NaN,4.0,"Animation,Comedy,Romance"
3,tt0000004,short,Un bon bock,Un bon bock,0,1892.0,NaN,12.0,"Animation,Short"
4,tt0000005,short,Blacksmith Scene,Blacksmith Scene,0,1893.0,NaN,1.0,"Comedy,Short"
...,...,...,...,...,...,...,...,...,...
8799413,tt9916848,tvEpisode,Episode #3.17,Episode #3.17,0,2010.0,NaN,NaN,"Action,Drama,Family"
8799414,tt9916850,tvEpisode,Episode #3.19,Episode #3.19,0,2010.0,NaN,NaN,"Action,Drama,Family"
8799415,tt9916852,tvEpisode,Episode #3.20,Episode #3.20,0,2010.0,NaN,NaN,"Action,Drama,Family"
8799416,tt9916856,short,The Wind,The Wind,0,2015.0,NaN,27.0,Short


## Save to DB

In [42]:
db = sqlite3.connect("var/reviews.db")

In [43]:
title_info.to_sql('titles', if_exists='replace', index=False, chunksize=5000, con=db)

In [44]:
db.commit()

In [46]:
db.execute("create unique index i_titles_pk on titles (titleId)")
db.commit()

In [47]:
db.close()